In [26]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Input, Dense
from keras.models import Model, Sequential

## 1.- Preprocesamiento de los datos

In [4]:
fdata = pd.read_csv('data_histories.csv', index_col="dp_folio")
fdata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,locout5,locout10,locout20,locout30,rest_act,rest1,rest5,rest10,rest20,rest30
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,1,1,1,1,0,0,0,0,0,0
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,0,0,0,0,1,3,0,0,0,0
3,F,34,3,Int,Sec,5,1,1,0,0,...,1,-1,-1,-1,0,0,0,-1,-1,-1
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,0,0,0,-1,0,0,0,0,0,0
5,M,42,4,EM,Sec,3,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0


### Funciones para clusterizar/discretizar los datos

In [5]:
#Function to clusterize categories of a certain feature, and add the new clusterized feature as a new column
# Clusters should be an input of the form {cluster_A: {categories}, cluster_B: [categories]}
# Ex. obesity = {0:[1,2,3], 1:[4,5,6]}
def clusterizeDiscrete(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        cat = False
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if original_list[index] in clusters[cluster]:
                new_list.append(cluster)
                cat = True
        if cat == False:
            new_list.append("N")
    new_data[new_name] = new_list
    return new_data

# Function to clusterize categories of a certain continous feature, and add the new clusterized feature as a
# new column
# Clusters shoud be an input of the form {cluster_A: {lambdaFunction1}, cluster_B: lambdaFunction2}
def clusterizeContinuous(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if eval(clusters[cluster])(original_list[index]):
                new_list.append(cluster)
                break
    new_data[new_name] = new_list
    return new_data
        

# Set of auxiliary high-order functions that will evaluate the conditions to binarize a history
def lessThan(num):
    return lambda n: n < num

def lessQThan(num):
    return lambda n: n <= num

def greaterThan(num):
    return lambda n: n > num

def greaterQThan(num):
    return lambda n: n >= num

def between(num1, num2):
    return lambda n: n <= num2 and n >= num1

### Definición y creación de los clusters

In [6]:
# Clusterize between obese and non obese
obesity = {0:[1,2,3], 1:[4,5,6]}
ndata = clusterizeDiscrete("AIMC", obesity, "obesity", fdata)

# Clusterize between degrees of study (higher degree vs. non higher degree)
degree = {0:["Prim", "Sec", "Bach", "CarTec", "Otro"], 1:["Lic", "Mast", "Doc", "PDoc"]}
ndata = clusterizeDiscrete("id_gestud", degree, "hdegree", ndata)

# Clusterize excercise features
ejer = {"A":"greaterQThan(2.5)", "B":"lessThan(2.5)"}
ndata = clusterizeContinuous("ejer_act", ejer, "ejer0B", ndata)
ndata = clusterizeContinuous("ejer1", ejer, "ejer1B", ndata)
ndata = clusterizeContinuous("ejer5", ejer, "ejer5B", ndata)
ndata = clusterizeContinuous("ejer10", ejer, "ejer10B", ndata)
ndata = clusterizeContinuous("ejer20", ejer, "ejer20B", ndata)
ndata = clusterizeContinuous("ejer30", ejer, "ejer30B", ndata)

# Clusterize stress features
estres = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("estres_act", estres, "estres0B", ndata)
ndata = clusterizeDiscrete("estres1", estres, "estres1B", ndata)
ndata = clusterizeDiscrete("estres5", estres, "estres5B", ndata)
ndata = clusterizeDiscrete("estres10", estres, "estres10B", ndata)
ndata = clusterizeDiscrete("estres20", estres, "estres20B", ndata)
ndata = clusterizeDiscrete("estres30", estres, "estres30B", ndata)

# Clusterize weight features
peso = {"A":[1,2,3], "B":[4,5]}
ndata = clusterizeDiscrete("peso_act", peso, "peso0B", ndata)
ndata = clusterizeDiscrete("peso1", peso, "peso1B", ndata)
ndata = clusterizeDiscrete("peso5", peso, "peso5B", ndata)
ndata = clusterizeDiscrete("peso10", peso, "peso10B", ndata)
ndata = clusterizeDiscrete("peso20", peso, "peso20B", ndata)
ndata = clusterizeDiscrete("peso30", peso, "peso30B", ndata)

# Clusterize weight features
condi = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("condi_act", condi, "condi0B", ndata)
ndata = clusterizeDiscrete("condi1", condi, "condi1B", ndata)
ndata = clusterizeDiscrete("condi5", condi, "condi5B", ndata)
ndata = clusterizeDiscrete("condi10", condi, "condi10B", ndata)
ndata = clusterizeDiscrete("condi20", condi, "condi20B", ndata)
ndata = clusterizeDiscrete("condi30", condi, "condi30B", ndata)

# Clusterize health features
salud = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("salud_act", salud, "salud0B", ndata)
ndata = clusterizeDiscrete("salud1", salud, "salud1B", ndata)
ndata = clusterizeDiscrete("salud5", salud, "salud5B", ndata)
ndata = clusterizeDiscrete("salud10", salud, "salud10B", ndata)
ndata = clusterizeDiscrete("salud20", salud, "salud20B", ndata)
ndata = clusterizeDiscrete("salud30", salud, "salud30B", ndata)

# Clusterize job features
academic = {0:["Admin", "Asi", "Coo", "E", "ED", "EM", "Int", "Jef", "Lab", "Sec", "Tec", "Vig"], 1:["Acade", "Inv", "InvE"]}
ndata = clusterizeDiscrete("Apuesto", academic, "academic", ndata)

# Clusterize walking features
walking = {"A":"greaterQThan(1800.0)", "B":"lessThan(1800.0)"}
ndata = clusterizeContinuous("dis_dia", walking, "dis_dia0B", ndata)
ndata = clusterizeContinuous("dis1_dia", walking, "dis_dia1B", ndata)
ndata = clusterizeContinuous("dis5_dia", walking, "dis_dia5B", ndata)
ndata = clusterizeContinuous("dis10_dia", walking, "dis_dia10B", ndata)
ndata = clusterizeContinuous("dis20_dia", walking, "dis_dia20B", ndata)
ndata = clusterizeContinuous("dis30_dia", walking, "dis_dia30B", ndata)


# Clusterize age features
age = {0:"between(15.0, 28.0)", 1:"between(28.1, 40.0)", 2:"between(40.1, 60.0)", 3:"between(60.1, 90.0)"}
ndata = clusterizeContinuous("Aedad", age, "AedadC", ndata)

In [7]:
ndata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,salud20B,salud30B,academic,dis_dia0B,dis_dia1B,dis_dia5B,dis_dia10B,dis_dia20B,dis_dia30B,AedadC
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,A,A,0,B,B,A,N,N,N,2
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,A,A,0,A,B,B,B,A,B,1
3,F,34,3,Int,Sec,5,1,1,0,0,...,A,N,0,B,B,B,B,B,B,1
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,A,A,0,A,A,A,A,A,A,3
5,M,42,4,EM,Sec,3,1,2,2,2,...,N,N,0,B,B,B,N,N,N,2


### Creación de la matriz de variables (features)

In [8]:
# Build profile based on excercise, health and stress in the last 0, 1, 5 and 10 years
profiles_0_to_10 = ndata[["ejer0B", "salud0B", "estres0B", "ejer1B", "salud1B", "estres1B", "ejer5B", "salud5B", "estres5B", "ejer10B", "salud10B", "estres10B"]]
profiles_1_to_10 = ndata[["ejer1B", "salud1B", "estres1B", "ejer1B", "salud1B", "estres1B", "ejer5B", "salud5B", "estres5B", "ejer10B", "salud10B", "estres10B"]]

# Replace "A" with 1, and "B" with 0, in order to have binary values, and save this in the feature matrix X
X_0 = profiles_0_to_10.replace("A", 1).replace("B", 0).replace("N", 0)
X_1 = profiles_1_to_10.replace("A", 1).replace("B", 0).replace("N", 0)

In [9]:
X_0.head()

,ejer0B,salud0B,estres0B,ejer1B,salud1B,estres1B,ejer5B,salud5B,estres5B,ejer10B,salud10B,estres10B
dp_folio,,,,,,,,,,,,
1,0,1,1,0,1,0,0,1,1,0,1,0
2,0,1,0,0,1,0,1,0,1,1,1,0
3,0,0,1,0,0,1,0,1,1,1,1,1
4,0,1,0,0,1,1,1,1,0,1,1,0
5,0,1,0,0,1,0,0,0,0,0,0,1


## 2.- RBM

In [11]:
# Simple function to split a set between batches
def batch_iterator(X, y=None, batch_size=64):
    n_samples = X.shape[0]
    for i in np.arange(0, n_samples, batch_size):
        begin, end = i, min(i+batch_size, n_samples)
        if y is not None:
            yield X[begin:end], y[begin:end]
        else:
            yield X[begin:end]

In [12]:
# Classic Restricted Boltzmann Machine
class RBM :
    
    def __init__(self, num_visible, num_hidden, learning_rate, batch_size, num_epochs):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        
        self.W = np.random.normal(scale=0.1, size=(num_visible, num_hidden))
        self.a = np.zeros(num_visible)
        self.b = np.zeros(num_hidden)
    
    # Calculate the sigmoid of X 
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-1*X))
    
    # Sample the activations given a certain matrix of probabilities
    def sample(self, X):
        return X > np.random.random_sample(size=X.shape)
    
    # Perform a reconstruction of the input data X
    def gibbs_sample(self, X):
        # Positive phase: Calculate the activations of the hidden layer
        positive_hidden = self.sigmoid(X.dot(self.W) + self.b)
        hidden_states = self.sample(positive_hidden)
        # Negative phase: Given the activations of the hidden layer, reconstruct the states at the visible layer
        negative_visible = self.sigmoid(hidden_states.dot(self.W.T) + self.a)
        visible_states = self.sample(negative_visible)
        return visible_states
    
    # Get the hidden probabilities for a certain input
    def transform(self, X):
        return self.sigmoid(X.dot(self.W) + self.b)
    
    def train(self, X):
        
        # Define matrix to keep track of the training MSE
        self.training_errors = []
        
        for epoch in range(self.num_epochs):
            
            batch_errors = []
            
            for batch in batch_iterator(X, batch_size=self.batch_size):
                # Positive phase: Calculate the activations of the hidden layer
                positive_hidden_probs = self.sigmoid(batch.dot(self.W) + self.b)
                positive_hidden_states = self.sample(positive_hidden_probs)
                
                # Calculate vh_data using the positive hidden states activations, rather than their probabilities
                # as per Hinton (2010)
                vh_data = batch.T.dot(positive_hidden_probs)
                
                # Negative phase
                negative_visible_probs = self.sigmoid(positive_hidden_states.dot(self.W.T) + self.a)
                negative_visible_states = self.sample(negative_visible_probs)
                negative_hidden_probs = self.sigmoid(negative_visible_states.dot(self.W) + self.b)
                negative_hidden_states = self.sample(negative_hidden_probs)
                
                # Calculate vh_reconstruction using the negative hidden states activations
                vh_reconstruction = negative_visible_states.T.dot(negative_hidden_probs)
                
                # Update weights and biases
                self.W += self.learning_rate * (vh_data - vh_reconstruction)
                self.b += self.learning_rate * (positive_hidden_probs.sum(axis=0) - negative_hidden_probs.sum(axis=0))
                self.a += self.learning_rate * (batch.sum(axis=0) - negative_visible_probs.sum(axis=0))
                
                batch_errors.append(np.mean((batch - negative_visible_states) ** 2))
                                    
            self.training_errors.append(np.mean(batch_errors))
                

In [13]:
rbm = RBM(12, 100, 0.001, 10, 1000)

In [773]:
rbm.train(X_0.values)

In [605]:
#rbm.training_errors

In [804]:
print X_0.iloc[0].values
print rbm.transform(X_0.iloc[0])
print rbm.gibbs_sample(X_0.iloc[0])

[0 1 1 0 1 0 0 1 1 0 1 0]
[0.45563637 0.13447236 0.19599848 0.69979971 0.26675212 0.47944314
 0.82066918 0.26620582 0.16830225 0.27615749 0.31852053 0.91182707
 0.59581526 0.82594039 0.10816511 0.35821711 0.46560322 0.68274718
 0.57748348 0.85117388 0.29124058 0.27504063 0.89311576 0.39435244
 0.1252799  0.11534753 0.36626307 0.08998397 0.68530103 0.0843418
 0.50107446 0.30758271 0.7537875  0.73723264 0.55508418 0.27650186
 0.66677256 0.70901189 0.56133186 0.22978427 0.80693958 0.66646107
 0.28614596 0.63995837 0.6086813  0.30599607 0.71330057 0.24803107
 0.58190218 0.39482271 0.28608284 0.11921054 0.16616146 0.19904229
 0.42260831 0.35844555 0.05254014 0.55746517 0.33025267 0.23706873
 0.35378862 0.19924487 0.06199506 0.16720627 0.49220731 0.12386191
 0.88646991 0.17999417 0.23698486 0.39055666 0.37432712 0.93181658
 0.39689718 0.10958925 0.49049599 0.16214901 0.85866197 0.37485209
 0.84195246 0.15480324 0.03382075 0.06092177 0.57670153 0.4019607
 0.1326676  0.65488931 0.20044968 0.09

## 3.- CRBM

In [16]:
# Conditional / Temporally Restricted Boltzmann Machine
class CRBM :
    
    def __init__(self, num_visible, num_hidden, num_historic, learning_rate, batch_size, num_epochs):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.num_historic = num_historic
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        
        # Initialize weights and biases
        self.W = np.random.normal(scale=0.1, size=(num_visible, num_hidden))
        self.a = np.zeros(num_visible)
        self.b = np.zeros(num_hidden)
        
        # Initialize autoregreesive parameters
        self.A = np.random.normal(scale=0.01, size=(num_visible, num_historic))
        self.B = np.random.normal(scale=0.01, size=(num_hidden, num_historic))
    
    # Calculate the sigmoid of X 
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-1*X))
    
    # Sample the activations given a certain matrix of probabilities
    def sample(self, X):
        return X > np.random.random_sample(size=X.shape)
    
    # Perform a reconstruction of the input data X
    def gibbs_sample(self, X):
        
        # Separate input vector
        V = np.array(X.values[:, :3])
        H = np.array(X.values[:, 3:])

        # Positive phase

        # Calculate the contributions from the Historic layer
        dinamic_b = self.b + H.dot(self.B.T)

        # Calculate the activations of the Hidden layer
        positive_hidden = sigmoid(V.dot(self.W) + dinamic_b)
        hidden_states = sample(positive_hidden)

        # Negative phase

        #Calculate the contributions from the Historic layer
        dinamic_a = self.a + H.dot(self.A.T)
        
        # Calculate the activations of the Visible layer
        negative_visible = sigmoid(hidden_states.dot(self.W.T) + dinamic_a)
        visible_states = sample(negative_visible)
        return visible_states
    
    # Get the hidden probabilities for a certain input
    def transform(self, X):
        V = np.array(X.values[:, :3])
        H = np.array(X.values[:, 3:])
        dinamic_b = self.b + H.dot(self.B.T)
        return self.sigmoid(X.dot(self.W) + dinamic_b)
    
    def train(self, X):
        
        # Define matrix to keep track of the training MSE
        self.training_errors = []
        
        for epoch in range(self.num_epochs):
            
            batch_errors = []
            
            for batch in batch_iterator(X, batch_size=self.batch_size):
                
                # Separate input vector
                V = np.array(batch.values[:, :3])
                H = np.array(batch.values[:, 3:])
                
                
                # Positive phase
                # Calculate the contributions from the Historic layer
                dinamic_b = self.b + H.dot(self.B.T)
                
                # Calculate the activations of the hidden layer
                positive_hidden_probs = self.sigmoid(V.dot(self.W) + dinamic_b)
                positive_hidden_states = self.sample(positive_hidden_probs)
                
                # Calculate vh_data using the positive hidden probabilities, rather than their activations
                # as per Hinton (2010)
                vh_data = V.T.dot(positive_hidden_probs)
                
                # Calculate vH_data (H: Historic layer)
                vH_data = V.T.dot(H)
                # Calculate hH_data (H: Historic layer)
                hH_data = positive_hidden_states.T.dot(H)
                
                
                
                # Negative phase
                
                # Calculate the contributions from the Historic layer
                dinamic_a = self.a + H.dot(self.A.T)
                
                negative_visible_probs = self.sigmoid(positive_hidden_states.dot(self.W.T) + dinamic_a)
                negative_visible_states = self.sample(negative_visible_probs)
                negative_hidden_probs = self.sigmoid(negative_visible_states.dot(self.W) + dinamic_b)
                negative_hidden_states = self.sample(negative_hidden_probs)
                
                # Calculate vh_reconstruction using the negative hidden states probabilities
                vh_reconstruction = negative_visible_states.T.dot(negative_hidden_probs)
                
                # Calculate vH_reconstruction (H: Historic layer)
                vH_reconstruction = negative_visible_states.T.dot(H)
                # Calculate hH_reconstruction (H: Historic layer)
                hH_reconstruction = negative_hidden_states.T.dot(H)
                
                # Update weights and biases
                self.W += self.learning_rate * (vh_data - vh_reconstruction)
                self.b += self.learning_rate * (positive_hidden_probs.sum(axis=0) - negative_hidden_probs.sum(axis=0))
                self.a += self.learning_rate * (V.sum(axis=0) - negative_visible_probs.sum(axis=0))
                self.B += self.learning_rate * 0.01 * (hH_data - hH_reconstruction)
                self.A += self.learning_rate * 0.01 * (vH_data - vH_reconstruction)
                
                batch_errors.append(np.mean((V - negative_visible_states) ** 2))
            print "Epoch " + str(epoch) + " - MSE: " + str(np.mean(batch_errors))                        
            self.training_errors.append(np.mean(batch_errors))
                

In [758]:
crbm = CRBM(3, 100, 9, 0.001, 10, 2000)

In [759]:
crbm.train(X_0)

Epoch 0 - MSE: 0.45493827160493827
Epoch 1 - MSE: 0.46419753086419746
Epoch 2 - MSE: 0.44074074074074077
Epoch 3 - MSE: 0.46296296296296297
Epoch 4 - MSE: 0.43703703703703706
Epoch 5 - MSE: 0.45637860082304516
Epoch 6 - MSE: 0.4671810699588476
Epoch 7 - MSE: 0.45884773662551437
Epoch 8 - MSE: 0.4441358024691358
Epoch 9 - MSE: 0.444238683127572
Epoch 10 - MSE: 0.4525720164609053
Epoch 11 - MSE: 0.4435185185185185
Epoch 12 - MSE: 0.4472222222222222
Epoch 13 - MSE: 0.4488683127572017
Epoch 14 - MSE: 0.45092592592592595
Epoch 15 - MSE: 0.44362139917695464
Epoch 16 - MSE: 0.4371399176954732
Epoch 17 - MSE: 0.43888888888888894
Epoch 18 - MSE: 0.4409465020576131
Epoch 19 - MSE: 0.45493827160493827
Epoch 20 - MSE: 0.4381687242798354
Epoch 21 - MSE: 0.46141975308641975
Epoch 22 - MSE: 0.4248971193415637
Epoch 23 - MSE: 0.4402263374485596
Epoch 24 - MSE: 0.45596707818930043
Epoch 25 - MSE: 0.44259259259259265
Epoch 26 - MSE: 0.4203703703703705
Epoch 27 - MSE: 0.4419753086419754
Epoch 28 - MSE: 0

Epoch 233 - MSE: 0.34475308641975305
Epoch 234 - MSE: 0.35390946502057613
Epoch 235 - MSE: 0.3128600823045267
Epoch 236 - MSE: 0.34670781893004116
Epoch 237 - MSE: 0.34259259259259267
Epoch 238 - MSE: 0.3457818930041152
Epoch 239 - MSE: 0.34382716049382717
Epoch 240 - MSE: 0.339917695473251
Epoch 241 - MSE: 0.34403292181069955
Epoch 242 - MSE: 0.347119341563786
Epoch 243 - MSE: 0.34660493827160493
Epoch 244 - MSE: 0.3599794238683127
Epoch 245 - MSE: 0.3414609053497942
Epoch 246 - MSE: 0.34218106995884773
Epoch 247 - MSE: 0.3483539094650205
Epoch 248 - MSE: 0.3286008230452675
Epoch 249 - MSE: 0.3445473251028807
Epoch 250 - MSE: 0.3435185185185186
Epoch 251 - MSE: 0.3420781893004115
Epoch 252 - MSE: 0.34434156378600816
Epoch 253 - MSE: 0.33549382716049386
Epoch 254 - MSE: 0.3440329218106996
Epoch 255 - MSE: 0.35164609053497947
Epoch 256 - MSE: 0.33796296296296297
Epoch 257 - MSE: 0.33786008230452674
Epoch 258 - MSE: 0.35462962962962963
Epoch 259 - MSE: 0.3444444444444445
Epoch 260 - MSE:

Epoch 462 - MSE: 0.28343621399176955
Epoch 463 - MSE: 0.2729423868312757
Epoch 464 - MSE: 0.27983539094650206
Epoch 465 - MSE: 0.26131687242798357
Epoch 466 - MSE: 0.28569958847736626
Epoch 467 - MSE: 0.269238683127572
Epoch 468 - MSE: 0.2916666666666667
Epoch 469 - MSE: 0.275
Epoch 470 - MSE: 0.27273662551440325
Epoch 471 - MSE: 0.27170781893004115
Epoch 472 - MSE: 0.2760288065843622
Epoch 473 - MSE: 0.2761316872427983
Epoch 474 - MSE: 0.26224279835390946
Epoch 475 - MSE: 0.26985596707818926
Epoch 476 - MSE: 0.2566872427983539
Epoch 477 - MSE: 0.27448559670781886
Epoch 478 - MSE: 0.2643004115226337
Epoch 479 - MSE: 0.2691358024691358
Epoch 480 - MSE: 0.27006172839506176
Epoch 481 - MSE: 0.26779835390946505
Epoch 482 - MSE: 0.27150205761316876
Epoch 483 - MSE: 0.2823045267489712
Epoch 484 - MSE: 0.2708847736625514
Epoch 485 - MSE: 0.27242798353909464
Epoch 486 - MSE: 0.2598765432098765
Epoch 487 - MSE: 0.2643004115226337
Epoch 488 - MSE: 0.2699588477366255
Epoch 489 - MSE: 0.2683127572

Epoch 691 - MSE: 0.21594650205761315
Epoch 692 - MSE: 0.21265432098765433
Epoch 693 - MSE: 0.21059670781893003
Epoch 694 - MSE: 0.21419753086419752
Epoch 695 - MSE: 0.19907407407407407
Epoch 696 - MSE: 0.20555555555555557
Epoch 697 - MSE: 0.202880658436214
Epoch 698 - MSE: 0.2152263374485597
Epoch 699 - MSE: 0.21172839506172836
Epoch 700 - MSE: 0.21574074074074076
Epoch 701 - MSE: 0.20462962962962963
Epoch 702 - MSE: 0.19660493827160494
Epoch 703 - MSE: 0.20709876543209876
Epoch 704 - MSE: 0.20390946502057614
Epoch 705 - MSE: 0.20072016460905354
Epoch 706 - MSE: 0.20586419753086418
Epoch 707 - MSE: 0.2020576131687243
Epoch 708 - MSE: 0.20072016460905348
Epoch 709 - MSE: 0.2059670781893004
Epoch 710 - MSE: 0.2070987654320988
Epoch 711 - MSE: 0.2042181069958848
Epoch 712 - MSE: 0.20977366255144037
Epoch 713 - MSE: 0.2205761316872428
Epoch 714 - MSE: 0.21676954732510292
Epoch 715 - MSE: 0.19948559670781896
Epoch 716 - MSE: 0.20853909465020576
Epoch 717 - MSE: 0.20226337448559675
Epoch 718

Epoch 916 - MSE: 0.15329218106995887
Epoch 917 - MSE: 0.16954732510288067
Epoch 918 - MSE: 0.16059670781893004
Epoch 919 - MSE: 0.15761316872427983
Epoch 920 - MSE: 0.1580246913580247
Epoch 921 - MSE: 0.16872427983539098
Epoch 922 - MSE: 0.16419753086419753
Epoch 923 - MSE: 0.15802469135802472
Epoch 924 - MSE: 0.1740740740740741
Epoch 925 - MSE: 0.16162551440329218
Epoch 926 - MSE: 0.16851851851851854
Epoch 927 - MSE: 0.1637860082304527
Epoch 928 - MSE: 0.16316872427983536
Epoch 929 - MSE: 0.16543209876543213
Epoch 930 - MSE: 0.1668724279835391
Epoch 931 - MSE: 0.1609053497942387
Epoch 932 - MSE: 0.16718106995884774
Epoch 933 - MSE: 0.1551440329218107
Epoch 934 - MSE: 0.16851851851851854
Epoch 935 - MSE: 0.15987654320987657
Epoch 936 - MSE: 0.16512345679012347
Epoch 937 - MSE: 0.16172839506172848
Epoch 938 - MSE: 0.14794238683127572
Epoch 939 - MSE: 0.17294238683127575
Epoch 940 - MSE: 0.1564814814814815
Epoch 941 - MSE: 0.16069958847736632
Epoch 942 - MSE: 0.16141975308641973
Epoch 94

Epoch 1136 - MSE: 0.1347736625514403
Epoch 1137 - MSE: 0.13621399176954732
Epoch 1138 - MSE: 0.12355967078189299
Epoch 1139 - MSE: 0.13220164609053497
Epoch 1140 - MSE: 0.1478395061728395
Epoch 1141 - MSE: 0.14742798353909464
Epoch 1142 - MSE: 0.13425925925925924
Epoch 1143 - MSE: 0.12469135802469135
Epoch 1144 - MSE: 0.13816872427983537
Epoch 1145 - MSE: 0.1353909465020576
Epoch 1146 - MSE: 0.12201646090534977
Epoch 1147 - MSE: 0.1338477366255144
Epoch 1148 - MSE: 0.1241769547325103
Epoch 1149 - MSE: 0.12150205761316873
Epoch 1150 - MSE: 0.12921810699588476
Epoch 1151 - MSE: 0.13724279835390943
Epoch 1152 - MSE: 0.1406378600823045
Epoch 1153 - MSE: 0.13580246913580246
Epoch 1154 - MSE: 0.12983539094650204
Epoch 1155 - MSE: 0.13549382716049382
Epoch 1156 - MSE: 0.1302469135802469
Epoch 1157 - MSE: 0.13292181069958847
Epoch 1158 - MSE: 0.13436213991769547
Epoch 1159 - MSE: 0.14228395061728394
Epoch 1160 - MSE: 0.12685185185185183
Epoch 1161 - MSE: 0.12561728395061728
Epoch 1162 - MSE: 0

Epoch 1356 - MSE: 0.11358024691358024
Epoch 1357 - MSE: 0.11646090534979424
Epoch 1358 - MSE: 0.10174897119341564
Epoch 1359 - MSE: 0.11625514403292181
Epoch 1360 - MSE: 0.11574074074074074
Epoch 1361 - MSE: 0.11162551440329217
Epoch 1362 - MSE: 0.10987654320987655
Epoch 1363 - MSE: 0.11872427983539095
Epoch 1364 - MSE: 0.11522633744855966
Epoch 1365 - MSE: 0.11059670781893002
Epoch 1366 - MSE: 0.10956790123456789
Epoch 1367 - MSE: 0.11841563786008227
Epoch 1368 - MSE: 0.11646090534979425
Epoch 1369 - MSE: 0.1156378600823045
Epoch 1370 - MSE: 0.12160493827160493
Epoch 1371 - MSE: 0.11296296296296296
Epoch 1372 - MSE: 0.10833333333333332
Epoch 1373 - MSE: 0.10905349794238682
Epoch 1374 - MSE: 0.11059670781893002
Epoch 1375 - MSE: 0.1176954732510288
Epoch 1376 - MSE: 0.11347736625514403
Epoch 1377 - MSE: 0.11069958847736625
Epoch 1378 - MSE: 0.11502057613168723
Epoch 1379 - MSE: 0.1090534979423868
Epoch 1380 - MSE: 0.10113168724279836
Epoch 1381 - MSE: 0.10864197530864196
Epoch 1382 - MS

Epoch 1576 - MSE: 0.1045267489711934
Epoch 1577 - MSE: 0.10030864197530863
Epoch 1578 - MSE: 0.10185185185185183
Epoch 1579 - MSE: 0.09537037037037034
Epoch 1580 - MSE: 0.09866255144032922
Epoch 1581 - MSE: 0.1037037037037037
Epoch 1582 - MSE: 0.1030864197530864
Epoch 1583 - MSE: 0.09907407407407408
Epoch 1584 - MSE: 0.08374485596707816
Epoch 1585 - MSE: 0.09660493827160492
Epoch 1586 - MSE: 0.09372427983539093
Epoch 1587 - MSE: 0.09681069958847735
Epoch 1588 - MSE: 0.09989711934156377
Epoch 1589 - MSE: 0.1001028806584362
Epoch 1590 - MSE: 0.10092592592592592
Epoch 1591 - MSE: 0.09866255144032919
Epoch 1592 - MSE: 0.1008230452674897
Epoch 1593 - MSE: 0.09393004115226337
Epoch 1594 - MSE: 0.09773662551440329
Epoch 1595 - MSE: 0.09063786008230451
Epoch 1596 - MSE: 0.10020576131687242
Epoch 1597 - MSE: 0.10318930041152262
Epoch 1598 - MSE: 0.10380658436213991
Epoch 1599 - MSE: 0.09948559670781892
Epoch 1600 - MSE: 0.10483539094650204
Epoch 1601 - MSE: 0.09002057613168722
Epoch 1602 - MSE:

Epoch 1796 - MSE: 0.08909465020576131
Epoch 1797 - MSE: 0.0796296296296296
Epoch 1798 - MSE: 0.08888888888888886
Epoch 1799 - MSE: 0.09032921810699589
Epoch 1800 - MSE: 0.08806584362139916
Epoch 1801 - MSE: 0.08271604938271604
Epoch 1802 - MSE: 0.0920781893004115
Epoch 1803 - MSE: 0.08868312757201645
Epoch 1804 - MSE: 0.08600823045267489
Epoch 1805 - MSE: 0.09578189300411522
Epoch 1806 - MSE: 0.08559670781893002
Epoch 1807 - MSE: 0.0838477366255144
Epoch 1808 - MSE: 0.09465020576131687
Epoch 1809 - MSE: 0.08806584362139916
Epoch 1810 - MSE: 0.08508230452674896
Epoch 1811 - MSE: 0.09331275720164608
Epoch 1812 - MSE: 0.0889917695473251
Epoch 1813 - MSE: 0.07993827160493826
Epoch 1814 - MSE: 0.0926954732510288
Epoch 1815 - MSE: 0.09094650205761316
Epoch 1816 - MSE: 0.10051440329218106
Epoch 1817 - MSE: 0.08117283950617284
Epoch 1818 - MSE: 0.0868312757201646
Epoch 1819 - MSE: 0.08127572016460904
Epoch 1820 - MSE: 0.0912551440329218
Epoch 1821 - MSE: 0.08302469135802468
Epoch 1822 - MSE: 0

In [805]:
crbm.training_errors

[0.45493827160493827,
 0.46419753086419746,
 0.44074074074074077,
 0.46296296296296297,
 0.43703703703703706,
 0.45637860082304516,
 0.4671810699588476,
 0.45884773662551437,
 0.4441358024691358,
 0.444238683127572,
 0.4525720164609053,
 0.4435185185185185,
 0.4472222222222222,
 0.4488683127572017,
 0.45092592592592595,
 0.44362139917695464,
 0.4371399176954732,
 0.43888888888888894,
 0.4409465020576131,
 0.45493827160493827,
 0.4381687242798354,
 0.46141975308641975,
 0.4248971193415637,
 0.4402263374485596,
 0.45596707818930043,
 0.44259259259259265,
 0.4203703703703705,
 0.4419753086419754,
 0.4291152263374486,
 0.4356995884773663,
 0.45246913580246906,
 0.42973251028806586,
 0.43569958847736623,
 0.44074074074074077,
 0.44814814814814813,
 0.434670781893004,
 0.4310699588477366,
 0.42510288065843616,
 0.42530864197530877,
 0.4302469135802469,
 0.4244855967078188,
 0.41831275720164607,
 0.4198559670781893,
 0.4193415637860082,
 0.43014403292181075,
 0.4502057613168724,
 0.4237654320

## 4.- SRBM

In [17]:
# Socially Restricted Boltzmann Machine
class SRBM :
    
    def __init__(self, num_visible, num_hidden, num_historic, learning_rate, batch_size, num_epochs):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.num_historic = num_historic
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        
        # Initialize weights and biases
        self.W = np.random.normal(scale=0.1, size=(num_visible, num_hidden))
        self.a = np.zeros(num_visible)
        self.b = np.zeros(num_hidden)
        
        # Initialize autoregreesive parameters
        self.A = np.random.normal(scale=0.01, size=(num_visible, num_historic))
        self.B = np.random.normal(scale=0.01, size=(num_hidden, num_historic))
    
    # Calculate the sigmoid of X 
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-1*X))
    
    # Sample the activations given a certain matrix of probabilities
    def sample(self, X):
        return X > np.random.random_sample(size=X.shape)
    
    # Perform a reconstruction of the input data X
    def gibbs_sample(self, X):
        
        # Create Historic layer
        H = np.tile(self.historic, (len(X), 1))
        
        # Separate input vector
        V = np.array(X.values[:, :3])

        # Positive phase

        # Calculate the contributions from the Historic layer
        dinamic_b = self.b + H.dot(self.B.T)

        # Calculate the activations of the Hidden layer
        positive_hidden = sigmoid(V.dot(self.W) + dinamic_b)
        hidden_states = sample(positive_hidden)

        # Negative phase

        #Calculate the contributions from the Historic layer
        dinamic_a = self.a + H.dot(self.A.T)
        
        # Calculate the activations of the Visible layer
        negative_visible = sigmoid(hidden_states.dot(self.W.T) + dinamic_a)
        visible_states = sample(negative_visible)
        return visible_states
    
    # Get the hidden probabilities for a certain input
    def transform(self, X):
        # Separate input vector
        V = np.array(X.values[:, :3])
        # Create Historic layer
        H = np.tile(self.historic, (len(X), 1))
        dinamic_b = self.b + H.dot(self.B.T)
        return self.sigmoid(X.dot(self.W) + dinamic_b)
    
    def train(self, X):
        
        # Define matrix to keep track of the training MSE
        self.training_errors = []
        
        self.historic = []
        for id in X.values[:, 3:]:
            for value in id:
                self.historic.append(value)
        
        for epoch in range(self.num_epochs):
            
            batch_errors = []
            
            for batch in batch_iterator(X, batch_size=self.batch_size):
                
                # Separate input vector
                V = np.array(batch.values[:, :3])   
                
                # Create Historic layer
                H = np.tile(self.historic, (len(batch), 1))
                #print H.shape
                
                # Positive phase
                # Calculate the contributions from the Historic layer
                dinamic_b = self.b + H.dot(self.B.T)
                
                # Calculate the activations of the hidden layer
                positive_hidden_probs = self.sigmoid(V.dot(self.W) + dinamic_b)
                positive_hidden_states = self.sample(positive_hidden_probs)
                
                # Calculate vh_data using the positive hidden probabilities, rather than their activations
                # as per Hinton (2010)
                vh_data = V.T.dot(positive_hidden_probs)
                
                # Calculate vH_data (H: Historic layer)
                vH_data = V.T.dot(H)
                # Calculate hH_data (H: Historic layer)
                hH_data = positive_hidden_states.T.dot(H)
                
                
                
                # Negative phase
                
                # Calculate the contributions from the Historic layer
                dinamic_a = self.a + H.dot(self.A.T)
                
                negative_visible_probs = self.sigmoid(positive_hidden_states.dot(self.W.T) + dinamic_a)
                negative_visible_states = self.sample(negative_visible_probs)
                negative_hidden_probs = self.sigmoid(negative_visible_states.dot(self.W) + dinamic_b)
                negative_hidden_states = self.sample(negative_hidden_probs)
                
                # Calculate vh_reconstruction using the negative hidden states probabilities
                vh_reconstruction = negative_visible_states.T.dot(negative_hidden_probs)
                
                # Calculate vH_reconstruction (H: Historic layer)
                vH_reconstruction = negative_visible_states.T.dot(H)
                # Calculate hH_reconstruction (H: Historic layer)
                hH_reconstruction = negative_hidden_states.T.dot(H)
                
                # Update weights and biases
                self.W += self.learning_rate * (vh_data - vh_reconstruction)
                self.b += self.learning_rate * (positive_hidden_probs.sum(axis=0) - negative_hidden_probs.sum(axis=0))
                self.a += self.learning_rate * (V.sum(axis=0) - negative_visible_probs.sum(axis=0))
                self.B += self.learning_rate * 0.001 * (hH_data - hH_reconstruction)
                self.A += self.learning_rate * 0.001 * (vH_data - vH_reconstruction)
                #print dinamic_a
                #print H.dot(self.A.T).sum()
                #print dinamic_b
                
                batch_errors.append(np.mean((V - negative_visible_states) ** 2))
            print "Epoch " + str(epoch) + " - MSE: " + str(np.mean(batch_errors))                        
            self.training_errors.append(np.mean(batch_errors))
                

In [64]:
srbm = SRBM(3, 500, 9684, 0.001, 10, 100)

In [65]:
srbm.train(X_0)

Epoch 0 - MSE: 0.39732510288065837
Epoch 1 - MSE: 0.38796296296296295
Epoch 2 - MSE: 0.3731481481481481
Epoch 3 - MSE: 0.38014403292181065


KeyboardInterrupt: 

## 5.- Autoencoder

In [48]:
dim = X_0.shape[-1]
encoding_dim = 4

# Define the input layer
inputs = Input(shape=(dim,))

# Define the encoding layer (turns inputs into an encoded representation)
encoded = Dense(encoding_dim, activation='relu')(inputs)

# Define the decoding layer (turns the encoded representation into a decoded output)
decoded = Dense(dim, activation='sigmoid')(encoded)

# Define the autoencoder
autoencoder = Model(inputs, decoded)

# Define the encoder (this will be used for the classification purposes)
encoder = Model(inputs, encoded)

In [58]:
# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [59]:
# Train the autoencoder
autoencoder.fit(X_0, X_0, epochs=500, batch_size=10)

Epoch 1/500
1076/1076 [==============================] - 0s 25us/step - loss: 0.6798
Epoch 2/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.6430
Epoch 3/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.6080
Epoch 4/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.5681
Epoch 5/500
1076/1076 [==============================] - 0s 30us/step - loss: 0.5225
Epoch 6/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.4822
Epoch 7/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.4526
Epoch 8/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.4306
Epoch 9/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.4140
Epoch 10/500
1076/1076 [==============================] - 0s 29us/step - loss: 0.4007
Epoch 11/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.3901
Epoch 12/500
1076/1076 [==============================] - 0s 26

1076/1076 [==============================] - 0s 27us/step - loss: 0.2533
Epoch 97/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2530
Epoch 98/500
1076/1076 [==============================] - 0s 25us/step - loss: 0.2525
Epoch 99/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2522
Epoch 100/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2516
Epoch 101/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2511
Epoch 102/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2508
Epoch 103/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2503
Epoch 104/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2500
Epoch 105/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2496
Epoch 106/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2490
Epoch 107/500
1076/1076 [==============================] - 0

1076/1076 [==============================] - 0s 32us/step - loss: 0.2265
Epoch 191/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2267
Epoch 192/500
1076/1076 [==============================] - 0s 29us/step - loss: 0.2266
Epoch 193/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2264
Epoch 194/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2265
Epoch 195/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2262
Epoch 196/500
1076/1076 [==============================] - 0s 29us/step - loss: 0.2262
Epoch 197/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2262
Epoch 198/500
1076/1076 [==============================] - 0s 43us/step - loss: 0.2261
Epoch 199/500
1076/1076 [==============================] - 0s 30us/step - loss: 0.2260
Epoch 200/500
1076/1076 [==============================] - 0s 33us/step - loss: 0.2259
Epoch 201/500
1076/1076 [==============================] 

1076/1076 [==============================] - 0s 30us/step - loss: 0.2223
Epoch 285/500
1076/1076 [==============================] - 0s 29us/step - loss: 0.2222
Epoch 286/500
1076/1076 [==============================] - 0s 32us/step - loss: 0.2220
Epoch 287/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2223
Epoch 288/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2221
Epoch 289/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2221
Epoch 290/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2221
Epoch 291/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2220
Epoch 292/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2221
Epoch 293/500
1076/1076 [==============================] - 0s 29us/step - loss: 0.2221
Epoch 294/500
1076/1076 [==============================] - 0s 31us/step - loss: 0.2221
Epoch 295/500
1076/1076 [==============================] 

1076/1076 [==============================] - 0s 26us/step - loss: 0.2204
Epoch 379/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2203
Epoch 380/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2202
Epoch 381/500
1076/1076 [==============================] - 0s 33us/step - loss: 0.2202
Epoch 382/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2202
Epoch 383/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2204
Epoch 384/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2202
Epoch 385/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2203
Epoch 386/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2203
Epoch 387/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2202
Epoch 388/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2203
Epoch 389/500
1076/1076 [==============================] 

1076/1076 [==============================] - 0s 31us/step - loss: 0.2192
Epoch 473/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2194
Epoch 474/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2193
Epoch 475/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2192
Epoch 476/500
1076/1076 [==============================] - 0s 26us/step - loss: 0.2194
Epoch 477/500
1076/1076 [==============================] - 0s 27us/step - loss: 0.2193
Epoch 478/500
1076/1076 [==============================] - 0s 30us/step - loss: 0.2193
Epoch 479/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2193
Epoch 480/500
1076/1076 [==============================] - 0s 25us/step - loss: 0.2194
Epoch 481/500
1076/1076 [==============================] - 0s 29us/step - loss: 0.2192
Epoch 482/500
1076/1076 [==============================] - 0s 28us/step - loss: 0.2192
Epoch 483/500
1076/1076 [==============================] 

In [60]:
encoded_X = encoder.predict(X_0)
decoded_X = autoencoder.predict(X_0)

In [61]:
encoded_X

array([[ 8.549894 ,  8.201173 ,  6.5100365,  4.5465355],
       [ 4.6182914,  6.4596996,  5.888911 ,  4.7400823],
       [10.822655 ,  6.7201233,  5.7032237,  9.387111 ],
       ...,
       [ 5.1914496,  8.865306 ,  9.464438 ,  6.083227 ],
       [ 7.4173737,  7.4129496,  6.1309133,  3.362461 ],
       [11.037502 ,  9.183042 ,  8.235109 , 10.911389 ]], dtype=float32)

In [62]:
decoded_X

array([[0.01217344, 0.99346644, 0.5123414 , ..., 0.09875391, 0.98932016,
        0.1218917 ],
       [0.5164573 , 0.9580014 , 0.04775598, ..., 0.49454075, 0.91135323,
        0.08655549],
       [0.02095297, 0.10780223, 0.99884677, ..., 0.47401494, 0.93418264,
        0.2768042 ],
       ...,
       [0.66328216, 0.9803357 , 0.02693679, ..., 0.92786646, 0.9907569 ,
        0.00565449],
       [0.01354004, 0.9930161 , 0.18355952, ..., 0.06367269, 0.98509985,
        0.0940465 ],
       [0.10106573, 0.44928697, 0.9979075 , ..., 0.9099296 , 0.985078  ,
        0.09062151]], dtype=float32)

In [63]:
print decoded_X[0]
print X_0.iloc[0]

[0.01217344 0.99346644 0.5123414  0.00775977 0.99670583 0.832999
 0.00889827 0.99858946 0.36137763 0.09875391 0.98932016 0.1218917 ]
ejer0B       0
salud0B      1
estres0B     1
ejer1B       0
salud1B      1
estres1B     0
ejer5B       0
salud5B      1
estres5B     1
ejer10B      0
salud10B     1
estres10B    0
Name: 1, dtype: int64


## 6.- Predicción de Obesidad

In [66]:
# Dividir entre train y test
# Construir un clasificador (perceptron simple) con todos y comparar sus resultados:
# Entrenar todos los modelos. 5-fold validation y construir ROC de cada uno. (Ya no nos enfoquemos en los parámetros distintos)

In [96]:
# Libraries for ROC plots
from scipy import interp
from sklearn.metrics import roc_curve, auc

# Libraries for Cross Validation
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold

# Libraries for Logistic Regression
from sklearn.linear_model import LogisticRegression

In [108]:
# Build profile based on excercise, health and stress in the last 0, 1, 5 and 10 years
profiles_0_to_10_wO = ndata[["ejer0B", "salud0B", "estres0B", "ejer1B", "salud1B", "estres1B", "ejer5B", "salud5B", "estres5B", "ejer10B", "salud10B", "estres10B", "obesity"]]

# Replace "A" with 1, and "B" with 0, in order to have binary values, and save this in the feature matrix X
X_Full = profiles_0_to_10_wO.replace("A", 1).replace("B", 0).replace("N", 0)

In [137]:
X_Full.iloc[:, :-1].head()

,ejer0B,salud0B,estres0B,ejer1B,salud1B,estres1B,ejer5B,salud5B,estres5B,ejer10B,salud10B,estres10B
dp_folio,,,,,,,,,,,,
1,0,1,1,0,1,0,0,1,1,0,1,0
2,0,1,0,0,1,0,1,0,1,1,1,0
3,0,0,1,0,0,1,0,1,1,1,1,1
4,0,1,0,0,1,1,1,1,0,1,1,0
5,0,1,0,0,1,0,0,0,0,0,0,1


In [82]:
kf = KFold(n_splits = 5, shuffle=True)

In [136]:
for train_index, test_index in kf.split(X_Full):
    train = []
    test = []
    for index in train_index:
        train.append(X_Full.iloc[index])
    train = np.array(train)
    for index in test_index:
        test.append(X_Full.iloc[index])
    test = np.array(test)

In [149]:
#rbm.transform(train[:, :-1])
#X_Full.iloc[:, -1:].shape
train[:, -1:].flatten()

array([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,

In [ ]:
# Perform a 5-fold cross validation for several polynomial degrees, and plot the ROCs
def run_cv(X, chosen_model, folds, name):
    
    lr = LogisticRegression(penalty='l1', solver='liblinear', C=1.0)
    kf = KFold(n_splits=folds, shuffle=True)
    
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    j = 0
    for train_index, test_index in kf.split(data):
        train = []
        test = []
        for index in train_index:
            train.append(X.iloc[index])
        train = np.array(train)
        for index in test_index:
            test.append(X.iloc[index])
        test = np.array(test)
        lr.fit(chosen_model.transform(train[:, :-1]), train[:, -1:])
        #print lr.score(Xv, Yv)
        
        Yh = lr.predict(Xv)
        Yprob = lr.predict_proba(Xv)
        
        fpr, tpr, thresholds = roc_curve(Yv, Yprob[:,1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label="ROC fold %d (AUC=%0.2f)" % (j + 1, roc_auc))
        j = j + 1
        
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)

    plt.plot(mean_fpr, mean_tpr, lw=2, color='b', alpha=0.8, label='Mean ROC (AUC=%0.2f)' % (mean_auc))
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(name)
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
           label='Random', alpha=.8)
    plt.legend(loc="lower right")
    plt.show()

### Tests

In [246]:
np_rng = np.random.RandomState()
weights = 0.1 * np.random.rand(3, 2)
weights

array([[0.03158819, 0.01143351],
       [0.00914015, 0.06884196],
       [0.09950659, 0.04133751]])

In [295]:
y = np.array([0.2, 0.8])
y > np.random.random_sample(size=y.shape)

array([ True,  True])

In [312]:
rbm = RBM(12, 10, 0.1, 10, 1000)

In [326]:
rbm.gibbs_sample(X_0.iloc[0].values)

array([False,  True,  True, False,  True,  True, False,  True, False,
       False,  True, False])

In [602]:
#for batch in batch_iterator(X_0, batch_size=100):
#    print batch

In [692]:
X_0.values.sum(axis=0)

array([424, 631, 506, 465, 647, 552, 565, 813, 360, 593, 903, 207])

In [743]:
print X_0.values[:, :3].shape
print X_0.values[:, 3:6].shape

(1076, 3)
(1076, 3)


In [612]:
#X_0.iloc[:, 3:]

In [554]:
num_visible = 3
num_hidden = 10
num_historic = 9

# Initialize weights and biases
W = np.random.normal(scale=0.1, size=(num_visible, num_hidden))
a = np.zeros(num_visible)
b = np.zeros(num_hidden)
        
# Initialize autoregreesive parameters
A = np.random.normal(scale=0.01, size=(num_visible, num_historic))
B = np.random.normal(scale=0.01, size=(num_hidden, num_historic))

# Calculate the sigmoid of X 
def sigmoid(X):
    return 1 / (1 + np.exp(-1*X))
    
# Sample the activations given a certain matrix of probabilities
def sample(X):
    return X > np.random.random_sample(size=X.shape)

# Perform a reconstruction of the input data X
def gibbs_sample(X):
        
    # Separate input vector
    V = np.array(X.values[:, :3])
    H = np.array(X.values[:, 3:])
        
    # Positive phase
        
    # Calculate the contributions from the Historic layer
    dinamic_b = b + H.dot(B.T).sum(axis=0)
    print b
    print dinamic_b
        
    positive_hidden = sigmoid(V.dot(W) + dinamic_b)
    hidden_states = sample(positive_hidden)
        
    # Negative phase
        
    #Calculate the contributions from the Historic layer
    dinamic_a = a + H.dot(A.T).sum(axis=0)
        
    negative_visible = sigmoid(hidden_states.dot(W.T) + dinamic_a)
    visible_states = sample(negative_visible)
    return visible_states

In [603]:
#gibbs_sample(X_0)